### 从网盘导入数据

In [ ]:
!git clone https://github.com/AlexandaJerry/PPASR.git
%cd /content/PPASR

Cloning into 'PPASR'...
remote: Enumerating objects: 2275, done.
remote: Counting objects: 100% (473/473), done.
remote: Compressing objects: 100% (163/163), done.
remote: Total 2275 (delta 338), reused 417 (delta 309), pack-reused 1802
Receiving objects: 100% (2275/2275), 32.09 MiB | 28.75 MiB/s, done.
Resolving deltas: 100% (1629/1629), done.
/content/PPASR


In [ ]:
wav_dir = 'inputs'#@param {type:"string"}
wav_path = '/content/PPASR/' + wav_dir
textgrid_dir = 'outputs'#@param {type:"string"}
textgrid_path = '/content/PPASR/' + textgrid_dir
acoustic_model = '/content/PPASR/mandarin.zip'
dictionary = '/content/PPASR/mandarin_pinyin.txt'
%mkdir $wav_dir
%mkdir $textgrid_dir

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_name = 'data.zip'#@param {type:"string"}
file_path = "/content/drive/MyDrive/" + file_name
!unzip -j $file_path "*.wav" -d $wav_path
!unzip -j $file_path "*.txt" -d $wav_path

Archive:  /content/drive/MyDrive/data.zip
  inflating: /content/PPASR/inputs/aam0511_slice_0000.wav  
  inflating: /content/PPASR/inputs/aam0511_slice_0001.wav  
  inflating: /content/PPASR/inputs/aam0511_slice_0002.wav  
  inflating: /content/PPASR/inputs/aam0511_slice_0003.wav  
  inflating: /content/PPASR/inputs/aam0511_slice_0004.wav  
  inflating: /content/PPASR/inputs/aam0511_slice_0005.wav  
  inflating: /content/PPASR/inputs/aam0511_slice_0006.wav  
  inflating: /content/PPASR/inputs/aam0511_slice_0007.wav  
  inflating: /content/PPASR/inputs/aam0511_slice_0008.wav  
  inflating: /content/PPASR/inputs/aam0511_slice_0009.wav  
  inflating: /content/PPASR/inputs/aam0511_slice_0010.wav  
  inflating: /content/PPASR/inputs/aam0511_slice_0011.wav  
  inflating: /content/PPASR/inputs/aam0511_slice_0012.wav  
  inflating: /content/PPASR/inputs/aam0511_slice_0013.wav  
  inflating: /content/PPASR/inputs/aam0511_slice_0014.wav  
  inflating: /content/PPASR/inputs/aam0511_slice_0015.wav 

### 安装MFA
#### 安装无误会显示2.0.6

In [1]:
!pip install -q condacolab
import condacolab
condacolab.install_miniconda()
!conda env create -n aligner -f environment.yml

⏬ Downloading https://repo.anaconda.com/miniconda/Miniconda3-py38_4.12.0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:21
🔁 Restarting kernel...


In [8]:
!source activate aligner; \
mfa version

/bin/bash: /usr/local/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/usr/local/envs/aligner/lib/python3.10/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(
2.0.6


#### 对齐前检测有无超出词典范围的词出现

In [ ]:
## 加载MFA普通话词典检测有无超出词典范围的词出现
import glob
import os
import shutil
import wave

import librosa
import soundfile
import tqdm

wav_dir = 'inputs'
sliced_path = '/content/PPASR/' + wav_dir
dict_path = '/content/PPASR/mandarin_pinyin.txt'
with open(dict_path, 'r', encoding='utf8') as f:
    rules = [ln.strip().split('\t') for ln in f.readlines()]
dictionary = {}
phoneme_set = set()
for r in rules:
    phonemes = r[1].split()
    dictionary[r[0]] = phonemes
    phoneme_set.update(phonemes)

# Run checks
check_failed = False
covered = set()
phoneme_map = {}
for ph in sorted(phoneme_set):
    phoneme_map[ph] = 0

segment_pairs = []
sliced_filelist = glob.glob(f'{sliced_path}/*.wav', recursive=True)
for file in tqdm.tqdm(sliced_filelist):
    filename = os.path.basename(file)
    name_without_ext = filename.rsplit('.', maxsplit=1)[0]
    annotation = os.path.join(sliced_path, f'{name_without_ext}.txt')
    if not os.path.exists(annotation):
        print(f'No annotation found for \'{filename}\'!')
        check_failed = True
    with open(annotation, 'r', encoding='utf8') as f:
        syllables = f.read().strip().split()
    if not syllables:
        print(f'Annotation file \'{annotation}\' is empty!')
        check_failed = True
    else:
        oov = []
        for s in syllables:
            if s not in dictionary:
                oov.append(s)
            else:
                for ph in dictionary[s]:
                    phoneme_map[ph] += 1
                covered.update(dictionary[s])
        if oov:
            print(f'Syllable(s) {oov} not allowed in annotation file \'{annotation}\'')
            check_failed = True

        else:
          print('no out-of-vocabulary words found in annotation files')

100%|██████████| 87/87 [00:00<00:00, 1977.84it/s]

no out-of-vocabulary words found in annotation files
no out-of-vocabulary words found in annotation files
no out-of-vocabulary words found in annotation files
no out-of-vocabulary words found in annotation files
no out-of-vocabulary words found in annotation files
no out-of-vocabulary words found in annotation files
no out-of-vocabulary words found in annotation files
no out-of-vocabulary words found in annotation files
no out-of-vocabulary words found in annotation files
no out-of-vocabulary words found in annotation files
no out-of-vocabulary words found in annotation files
no out-of-vocabulary words found in annotation files
no out-of-vocabulary words found in annotation files
no out-of-vocabulary words found in annotation files
no out-of-vocabulary words found in annotation files
no out-of-vocabulary words found in annotation files
no out-of-vocabulary words found in annotation files
no out-of-vocabulary words found in annotation files
no out-of-vocabulary words found in annotation



---



## 短音频对齐示范

---


### mfa align命令必须输入的参数包括四个路径：
#### 01音频和转写的路径 02词典的路径 03声学模型的路径 04输出textgrid的路径
#### 我们的词典是拼音词典mandarin_pinyin.txt
#### 我们的声学模型路径是mandarin.zip

---


### mfa align命令可以自由选择输入的参数包括：
#### --clean → 清除上次对齐时保留的缓存文件
#### -s number → 文件名中的前多少个字母为发音人的ID
#### 例如前两位文件名是发音人的ID，我们会用-s 2进行发音人的区分
#### --beam number → 对齐搜索的精度默认数字为10
#### 正常情况不需要修改beam，如果无法对齐则需要增大beam
#### --overwrite → 覆盖上次对齐好的Textgrid

---

### 示范命令：
!mfa align --clean -s 2 wav_txt_dir dictionary_path acoustic_model_dir textgrids_dir --beam 10 --overwrite


In [ ]:
# Slicer arguments
speaker_id = 6 #@param {type:"number"}
beam_number = 100 #@param {type:"number"}
wav_dir = 'inputs'
wav_path = '/content/PPASR/' + wav_dir
textgrid_dir = 'outputs'
textgrid_path = '/content/PPASR/' + textgrid_dir
acoustic_model = '/content/PPASR/mandarin.zip'
dictionary = '/content/PPASR/mandarin_pinyin.txt'

!source activate aligner; \
mfa align --clean -s $speaker_id $wav_path $dictionary $acoustic_model $textgrid_path --beam $beam_number --overwrite

/usr/local/envs/aligner/lib/python3.10/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(
INFO - Setting up corpus information...
INFO - Loading corpus from source files...
100% 87/87 [00:01<00:00, 81.81it/s]
INFO - Found 4 speakers across 87 files, average number of utterances per speaker: 21.75
INFO - Initializing multiprocessing jobs...
INFO - Creating corpus split for feature generation...
INFO - Generating base features (mfcc)...
INFO - Generating MFCCs...
 92% 80/87 [00:06<00:00, 13.26it/s]
INFO - Calculating CMVN...
INFO - Creating corpus split with features...
INFO - Compiling training graphs...
100% 87/87 [00:01<00:00, 62.91it/s] 
INFO - Performing first-pass alignment...
INFO - Generating

In [ ]:
from google.colab import files
zip_name_you_wish = 'textgrid.zip'#@param {type:"string"}
zip_path = '/content/PPASR/' + zip_name_you_wish
!zip -j $zip_path $textgrid_path/*
!cp -i $zip_path "/content/drive/MyDrive/"
print("files have been saved to MyDrive")
print("files would be automatically downloaded after several seconds")
files.download(zip_path)

  adding: aam0511_slice_0000.TextGrid (deflated 87%)
  adding: aam0511_slice_0001.TextGrid (deflated 87%)
  adding: aam0511_slice_0002.TextGrid (deflated 86%)
  adding: aam0511_slice_0003.TextGrid (deflated 86%)
  adding: aam0511_slice_0004.TextGrid (deflated 87%)
  adding: aam0511_slice_0005.TextGrid (deflated 87%)
  adding: aam0511_slice_0006.TextGrid (deflated 87%)
  adding: aam0511_slice_0007.TextGrid (deflated 87%)
  adding: aam0511_slice_0008.TextGrid (deflated 87%)
  adding: aam0511_slice_0009.TextGrid (deflated 88%)
  adding: aam0511_slice_0010.TextGrid (deflated 87%)
  adding: aam0511_slice_0011.TextGrid (deflated 88%)
  adding: aam0511_slice_0012.TextGrid (deflated 88%)
  adding: aam0511_slice_0013.TextGrid (deflated 88%)
  adding: aam0511_slice_0014.TextGrid (deflated 85%)
  adding: aam0511_slice_0015.TextGrid (deflated 86%)
  adding: aam0511_slice_0016.TextGrid (deflated 87%)
  adding: aam0511_slice_0017.TextGrid (deflated 88%)
  adding: aam0511_slice_0018.TextGrid (deflate

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>